In [4]:
%load_ext autoreload
%autoreload 2

import re
import sys
import nltk   
import time
import glob
import pathlib
import requests
import unicodedata
import numpy as np
import pandas as pd

#from utils import *
from bs4 import BeautifulSoup
from html.parser import HTMLParser
from tqdm import tqdm_notebook as tqdm

from sklearn.pipeline import Pipeline
from sklearn.naive_bayes import MultinomialNB
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer

agent = { 'User-Agent': 'Mozilla/5.0 (Windows NT 6.1; WOW64; rv:50.0) Gecko/20100101 Firefox/50.0',
          'Accept-Language': 'pt-BR'}

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


## Most frequent Attributes

In [5]:
# Low case only, remenber toremove non-ascii
most_frequents = {
    'price':['preço', 'preco', ],
    'model':['modelo'],
    'ram':['memoria ram', 'ram', 'memória RAM'],
    'hd':['armazenamento interno', 'memória interna', 'memoria interna', 'interna'],
    'screen':['tamanho da tela', 'tela', 'tamanho do display', 'display', 'tamanho']
}